## Load available keys

In [2]:
import ooiod

In [4]:
key_file = '/home/jovyan/repos/ooi-opendata/secrets/daxsoule.pickle'
keys = ooiod.secrets.load_keys(key_file)

You have:
    botpt


## Download a NetCDF file from OOI results server

In [2]:
netcdf_url = 'https://opendap.oceanobservatories.org/thredds/catalog/ooi/dax.soule@qc.cuny.edu/20191026T023346674Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/catalog.html?dataset=ooi/dax.soule@qc.cuny.edu/20191026T023346674Z-RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample/deployment0001_RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_20191014T120000-20191014T235959.950000.nc'
netcdf_filename = netcdf_url.split('/')[-1]

In [3]:
import wget

In [4]:
wget.download(netcdf_url);

## Write the original NetCDF file into Azure blob

In [5]:
from azure.storage.blob import BlockBlobService

In [6]:
if 'botpt' in keys:
    botpt_service = BlockBlobService('ooiopendata', sas_token = keys['botpt'])
else:
    raise Exception('Botpt container key not available')

NameError: name 'keys' is not defined